In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models
import collections
import math
import sys
sys.path.append("/home/rishi/Projects/Pyramid-fuse/")
# sys.setrecursionlimit(10000000)
import Utils
from Utils.CubePad import CustomPad
image_path = '/home/rishi/Projects/Pyramid-fuse/Sample.jpg'

In [2]:
import numpy as np

In [3]:
from PIL import Image
import cv2

In [4]:
img = np.asarray(Image.open(image_path), dtype=np.float32)
img=img.transpose(2,0,1)
img=np.expand_dims(img,0)
img.shape

(1, 3, 1024, 2048)

In [5]:
class CETransform(nn.Module):
    def __init__(self):
        super(CETransform, self).__init__()
        equ_h = [512, 128, 64, 32, 16]
        cube_h = [256, 64, 32, 16, 8]

        self.c2e = dict()
        self.e2c = dict()

        for h in equ_h:
            a = Utils.Equirec2Cube(1, h, h*2, h//2, 90)
            self.e2c['(%d,%d)' % (h, h*2)] = a

        for h in cube_h:
            a = Utils.Cube2Equirec(1, h, h*2, h*4)
            self.c2e['(%d)' % (h)] = a

    def E2C(self, x):
        print(x.shape)
        [bs, c, h, w] = x.shape
        key = '(%d,%d)' % (h, w)
        print(key)
        assert key in self.e2c
        return self.e2c[key].ToCubeTensor(x)

    def C2E(self, x):
        print(x.shape)
        [bs, c, h, w] = x.shape
        key = '(%d)' % (h)
        assert key in self.c2e and h == w
        return self.c2e[key].ToEquirecTensor(x)

    def forward(self, equi, cube):
        return self.e2c(equi), self.c2e(cube)


In [6]:
mypad = getattr(Utils.CubePad, 'SpherePad')

In [7]:
ce = CETransform()

In [8]:
img1 = torch.rand((1,3,512,1024)).cuda()

In [9]:
img1.shape

torch.Size([1, 3, 512, 1024])

In [10]:
test = ce.E2C(img1)

torch.Size([1, 3, 512, 1024])
(512,1024)


/home/rishi/.local/lib/python3.6/site-packages/torch/nn/functional.py:3226: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and affine_grid behavior has changed "


In [11]:
test2 = ce.C2E(test)

torch.Size([6, 3, 256, 256])
